# Bayesian Personalized Ranking
* See the corresponding file in `../TrainingAlphas` for more details

In [1]:
source = "BPR";

In [2]:
using Flux
using LinearAlgebra
import NBInclude: @nbinclude
import Random: shuffle
@nbinclude("Alpha.ipynb");

In [3]:
function pairwise_evaluate(m, X, i)
    # for item i, computes pairwise prefernce against all other items 
    m(vcat(repeat(X[:, i], 1, num_items()), X))
end

function get_pairwise_preferences()
    params = read_params(source)

    X = []
    for alpha in params["hyp"].features
        push!(X, read_recommendee_alpha(alpha, "all").rating)
    end
    X = reduce(hcat, X)
    X = collect(X')

    m = params["m"]
    Pt = Array{Float32}(undef, num_items(), num_items())
    @tprogress Threads.@threads for i = 1:num_items()
        Pt[:, i] = pairwise_evaluate(m, X, i)
    end
    collect(Pt')
end;

In [4]:
function get_binary_preferences()
    P = get_pairwise_preferences()
    Q = Array{Float32}(undef, size(P)...)
    # we shuffle the item order to evenly distribute the work
    @tprogress Threads.@threads for j in shuffle(1:num_items())
        for i = 1:j-1
            Q[i, j] = (P[i, j] > P[j, i]) ? 1 : -1
            Q[j, i] = -Q[i, j]
        end
    end
    Q
end;

In [5]:
function inital_state(P)
    sortperm(vec(sum(P, dims = 2)), rev = true)
end;

In [6]:
function score(r, P)
    # a negative score is good
    s = 0
    @tprogress Threads.@threads for j in shuffle(1:num_items())
        for i = 1:j-1
            s += -P[r[i], r[j]]
        end
    end
    s
end;

In [7]:
function anneal(r, P, iters, T)
    # peforms simulated annealing with a constant temperature
    # the action space is swapping consecutive pairs
    r = copy(r)
    ds = 0
    for _ = 1:iters
        # consider swapping r[i] with r[i+1]
        i = rand(1:num_items()-1)
        delta = P[r[i], r[i+1]] # we should swap if this is negative
        swap = (delta < 0) || (rand() < exp(-delta / T))
        if swap
            r[i], r[i+1] = r[i+1], r[i]
            ds += delta
        end
    end
    r, ds
end;

In [8]:
function anneal(r, P)
    # performs simulated annealing with a temperature schedule
    T = 1
    s = 0
    while T > 1e-6
        rt, ds = anneal(r, P, 100000, T)
        if ds >= 0
            T *= 0.5
        else
            r = rt
            s += ds
        end
    end
    r, s
end;

In [9]:
function multistart_anneal(r, P, iters)
    s = 0
    for _ = 1:iters
        r, ds = anneal(r, P)
        s += ds
        if ds == 0
            break
        end
    end
    r, s
end;

In [10]:
function compute_alpha()
    P = get_binary_preferences()
    ranking, _ = multistart_anneal(inital_state(P), P, 100)
    alpha = Array{Float32}(undef, num_items())
    for i = 1:num_items()
        alpha[ranking[i]] = i
    end
    write_recommendee_alpha(alpha, source)
end;

In [11]:
compute_alpha();

[ Warning: 20220612 03:14:36 type Main.Hyperparams does not exist in workspace; reconstructing
Progress: 100%|███████████████████████████| Time: 0:00:06 ( 2.93 ms/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 0.26 ms/it)
